### Proyecto Sistemas Recomendadores


1.   Sofía Olmedo
2.   Sebastián Veliz
3.   Matías Aedo

### Instalación de librerias y carga de datos

In [1]:
!pip3 install pyreclab --upgrade
!pip3 install implicit --upgrade
!pip install recmetrics


from   IPython.display import clear_output
import pandas as pd
import pyreclab
import seaborn as sns
import numpy as np
import scipy.sparse as sparse
import matplotlib.pyplot as plt
import os
import glob
import implicit
import random
from sklearn.decomposition import PCA
from sklearn.metrics import pairwise_distances
import heapq
import recmetrics
clear_output()

In [2]:
!wget https://www.dropbox.com/s/a3f7ipoad408x6w/product_data.csv
!wget https://www.dropbox.com/s/78ddkyxjz55zwdr/sales.csv
!wget https://www.dropbox.com/s/t3t028kz7qwcbem/tags.csv
!wget https://www.dropbox.com/s/xz4ekirr7h8te5t/popularity.csv
!wget https://www.dropbox.com/s/x2gr43vbnhsaqtt/producttag.csv

--2022-12-09 13:45:48--  https://www.dropbox.com/s/a3f7ipoad408x6w/product_data.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.85.18, 2620:100:6035:18::a27d:5512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.85.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/a3f7ipoad408x6w/product_data.csv [following]
--2022-12-09 13:45:49--  https://www.dropbox.com/s/raw/a3f7ipoad408x6w/product_data.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc3e7816e3dc48ae41684a57c0c9.dl.dropboxusercontent.com/cd/0/inline/ByRVgA_V-qd9oW7tfiKImfgdgRTlqqAsePRtLfoFau_p9Je0lN_NXfrR2MyO0XKEJbsGVwTF8jB7q87QCgNygocTzM61aO_RZare0cvL-Lik37UB95SwnNLGlQeC6pW-2K-JW4UC5WYQ5mVB_-p4uyzn0MyLPK-z8I-Pp4igtmSiqA/file# [following]
--2022-12-09 13:45:49--  https://uc3e7816e3dc48ae41684a57c0c9.dl.dropboxusercontent.com/cd/0/inline/ByRVgA_V-qd9oW7tfiKImfgdgRTlqqAsePRtLfoFau_p9Je0lN_NXfrR2MyO0XKEJ

Lectura de datos

In [3]:
products = pd.read_csv('product_data.csv')[['_id','price','referencePrice','name','BrandId','customAttributes','color','pattern', 'detailsScore','composition','storeDescription']]
popularity_insights= pd.read_csv('popularity.csv')
sales = pd.read_csv('sales.csv')
tags = pd.read_csv('tags.csv')
product_tag = pd.read_csv('producttag.csv')


/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (30,32) have mixed types.Specify dtype option on import or set low_memory=False.
  # an InputRejected.  Short-circuit in this case so that we


In [4]:
popularity_insights.head()

,_id,ProductId,clicks,impressions,type,createdAt,updatedAt,favorites
0,215231,7137,3,4,historical-ctr,2022-06-15 02:10:00.545000 +00:00,2022-08-27 16:10:00.663000 +00:00,0
1,73157,8963,26,1421,historical-ctr,2022-02-07 20:11:11.513000 +00:00,2022-02-13 22:11:10.628000 +00:00,0
2,94407,9394,77,4351,historical-ctr,2022-02-28 17:10:00.496000 +00:00,2022-08-04 01:10:00.551000 +00:00,1
3,84093,10314,5,6,historical-ctr,2022-02-20 15:10:00.767000 +00:00,2022-09-25 00:10:01.061000 +00:00,0
4,73978,12635,8,7,historical-ctr,2022-02-08 15:10:36.039000 +00:00,2022-07-11 23:10:00.495000 +00:00,0


## Análisis de las tablas

### Products

In [5]:
products.head()

,_id,price,referencePrice,name,BrandId,customAttributes,color,pattern,detailsScore,composition,storeDescription
0,7137,5990,19990,Polera Mango,1777,{},NaN,NaN,0,[],Marca: Mango\nMedidas:\nCondición: Sensacional...
1,8963,4990,12990,Jeans Forever 21,680,{},NaN,NaN,0,[],Marca: Forever 21\nMedidas:\nCondición: Sensac...
2,9394,4546,39190,Pantalón Bisou Bisou,3566,"{""1"":"""",""2"":"""",""3"":"""",""8"":"""",""9"":"""",""10"":"""",""1...",NaN,NaN,0,[],Marca: Bisou Bisou\nMedidas:\nCondición: Sensa...
3,10314,2794,18990,Chaleco Zara,640,"{""1"":"""",""2"":"""",""3"":"""",""4"":"""",""5"":"""",""7"":"""",""8""...",NaN,NaN,0,[],Marca: Zara\nMedidas:\nCondición: Sensacional\...
4,11107,26000,140000,Vestido Caro Criado,2527,{},NaN,NaN,0,[],Marca: Caro Criado\nMedidas:\nCondición: Sensa...


In [6]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39152 entries, 0 to 39151
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   _id               39152 non-null  int64 
 1   price             39152 non-null  int64 
 2   referencePrice    39152 non-null  int64 
 3   name              39152 non-null  object
 4   BrandId           39152 non-null  int64 
 5   customAttributes  39152 non-null  object
 6   color             9192 non-null   object
 7   pattern           9192 non-null   object
 8   detailsScore      39152 non-null  int64 
 9   composition       39152 non-null  object
 10  storeDescription  39033 non-null  object
dtypes: int64(5), object(6)
memory usage: 3.3+ MB


Primero, se analiza el precio al que se vende cada prenda para hacernos una idea de la clase de productos que se venden en la tienda. También se hace lo mismo con el referencePrice, que es el precio original calculado por la tienda.

In [7]:
price_mean = products['price'].value_counts(bins=[0000,5000,10000,15000,20000,25000,30000,35000,40000,45000], sort=False)
price_mean.plot(kind='bar')

ImportError: ignored

<Figure size 432x288 with 1 Axes>

In [8]:
r_price_mean = products['referencePrice'].value_counts(bins=[5000,10000,15000,20000,25000,30000,35000,40000,45000,50000,55000,60000,65000,70000,75000,800000], sort=False)
r_price_mean.plot(kind='bar')

ImportError: ignored

<Figure size 432x288 with 1 Axes>

Otra columna que se puede analizar facilmente es la de BrandId, la que nos dará la información sobre la distribución de marcas dentro de las prendas vendidas.

In [9]:
products['BrandId'].value_counts()[:25].plot(kind='barh', figsize=(5,5))

ImportError: ignored

<Figure size 360x360 with 1 Axes>

Al buscar estos ids en la base de datos, se puede ver que los productos con las 10 marcas más populares (en orden de más popular a menos popuular) son:
- Sin marca
- Zara
- H&M
- Forever 21
- Mango
- Sybilla
- Basement
- Opposite
- Shein
- Umbrale

Las otras columnas son dificiles de analizar por dos razones:
- El sparcity de sus valores como se puede ver en la celda de info
- El formato de las columnas no es amigable

### Sales

In [10]:
sales.head()

,sale_id,createdAt,client_id,product_list
0,125639,2022-10-13 06:59:49.253000 +00:00,78484,"{751922,780297,807277,924468,806791,859118,818..."
1,125626,2022-10-13 02:56:44.232000 +00:00,118131,"{565103,977773,775335,963132,880986,866684,952..."
2,125614,2022-10-13 01:25:52.991000 +00:00,115448,"{949847,977607,898317,929204,957821,749183,972..."
3,125605,2022-10-13 00:21:05.584000 +00:00,9020,"{884365,737438,806086,849056,634504,987273,755..."
4,125601,2022-10-12 23:12:48.718000 +00:00,118079,"{768824,738033,791608,770286,915205,960971,783..."


In [11]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   sale_id       3000 non-null   int64 
 1   createdAt     3000 non-null   object
 2   client_id     3000 non-null   int64 
 3   product_list  3000 non-null   object
dtypes: int64(2), object(2)
memory usage: 93.9+ KB


Esta tabla sólo se usa para unir clientes con sus respectivas compras.

### Tags

Se elimina la columna de "Value", ya que indica la misma información que "Translation", solo que en otro formato. Y cambiamos el nombre de esta columna.

In [12]:
del tags["Value"]
tags.rename(columns = {'Translation':'detail'}, inplace = True)

In [13]:
tags.head()

,TagId,detail,TagType
0,25,Maternal,additional_category
1,43,Como Nuevo,condition
2,44,Con Etiqueta,condition
3,45,Con Detalles,condition
4,1,Superior,garment_class


In [14]:
tags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1115 entries, 0 to 1114
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   TagId    1115 non-null   int64 
 1   detail   1101 non-null   object
 2   TagType  1115 non-null   object
dtypes: int64(1), object(2)
memory usage: 26.3+ KB


In [15]:
popularity_insights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32666 entries, 0 to 32665
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   _id          32666 non-null  int64 
 1   ProductId    32666 non-null  int64 
 2   clicks       32666 non-null  int64 
 3   impressions  32666 non-null  int64 
 4   type         32666 non-null  object
 5   createdAt    27994 non-null  object
 6   updatedAt    32562 non-null  object
 7   favorites    32666 non-null  int64 
dtypes: int64(5), object(3)
memory usage: 2.0+ MB


### ProductTags

In [16]:
product_tag.head()

,ProductId,TagId
0,7137,324
1,7137,1001
2,7137,1032
3,7137,1039
4,7137,1040


## Limpieza de datos

### Reducción de datos
Para optimizar el entrenamiento, se tomará una muestra de ventas y se armará 

In [17]:
present_clients = sales.client_id.unique()
present_clients.sort()
print(len(present_clients))

1399


Vamos a sacar una muestra de 200 clientes, de las que vamos a basar los otros dataframes en la información que se tenga de estos.

In [18]:
used_clients = np.random.choice(present_clients, size=200)
used_sales = sales[sales['client_id'].isin(used_clients)]
used_products_set = set()
for sale in used_sales.iterrows():
  product_list = sale[1].product_list.strip().lstrip('{').rstrip('}').split(',')
  for product in product_list:
    used_products_set.add(int(product))

used_products = products[products['_id'].isin(used_products_set)]
used_popularity_insights = popularity_insights[popularity_insights['ProductId'].isin(used_products_set)]
used_product_tag = product_tag[product_tag['ProductId'].isin(used_products_set)]

### Obtención de tallas


In [19]:
import json
atributos = products[["_id", "customAttributes", "storeDescription"]]
tallas = dict()
for _, row in atributos.iterrows():
  dicc = json.loads(row["customAttributes"])
  llaves = list(dicc.keys())
  if '2' in llaves:
    if dicc['2'] != '':
      tallas[row["_id"]] = dicc['2'].lower().replace(" estimada",'').replace("eur ",'').replace(" talla aprox",'').replace(" aproximada",'')
    elif isinstance(row["storeDescription"], str) and row["storeDescription"] != '':
      desc = row["storeDescription"].split('\n')
      desc = list(map(lambda x: x.split(': '), desc))
      for tupla in desc:
        if tupla[0] == 'Talla':
          tallas[row["_id"]] = tupla[1].lower().replace(" estimada",'').replace("eur ",'').replace(" talla aprox",'').replace(" aproximada",'')
print(len(tallas))

768


In [20]:
# tallas

### Most popular

In [21]:
def takeFirst(elem):
    return elem[0]
dict_popular = {}
lista_popular = []
for index, row in popularity_insights.iterrows():
  dict_popular[row['ProductId']] = row['favorites']
  lista_popular.append([row['favorites'], row['ProductId']])
lista_popular.sort(key=takeFirst, reverse=True)
print(lista_popular)

[[142, 590742], [75, 838042], [59, 849481], [57, 858669], [54, 907085], [54, 951827], [53, 939496], [52, 835390], [52, 965455], [49, 909871], [46, 885694], [45, 626663], [45, 883915], [44, 805397], [44, 835043], [44, 885597], [44, 885615], [44, 912209], [43, 892173], [43, 956776], [42, 790935], [41, 811961], [41, 832197], [41, 866487], [41, 909490], [41, 916886], [40, 903296], [39, 798404], [39, 834530], [39, 893765], [39, 967922], [39, 989368], [38, 773057], [38, 899436], [38, 935391], [37, 518372], [37, 826098], [37, 848974], [37, 881932], [37, 909093], [37, 929156], [37, 930102], [37, 951939], [36, 847910], [36, 859099], [36, 868601], [36, 872630], [36, 890238], [36, 913958], [36, 980489], [35, 850724], [35, 878483], [35, 894805], [35, 915589], [34, 748834], [34, 857273], [34, 859966], [34, 875430], [34, 879960], [34, 898680], [34, 914140], [34, 929635], [34, 972020], [33, 660728], [33, 661896], [33, 799659], [33, 801708], [33, 811105], [33, 842588], [33, 900294], [33, 912450], [33,

In [22]:
dict_sales = {}
for index, row in used_sales.iterrows():
  # print(row['product_list'][1:len(row['product_list'])-1].split(','))
  # dict_sales[row['client_id']] = row['product_list'][1:len(row['product_list'])-1].split(',')
  if row['client_id'] in dict_sales.keys():
    aux = dict_sales[row['client_id']] + row['product_list'][1:len(row['product_list'])-1].split(',')
    dict_sales[row['client_id']] = aux
  else:
    dict_sales[row['client_id']] = row['product_list'][1:len(row['product_list'])-1].split(',')
print(dict_sales)

{114404: ['803352', '673048', '786713', '798351', '650122', '703497', '812110', '771983', '642394', '711293', '806082', '822750', '863217', '868386', '663626', '722687', '800482', '850510', '862323', '841856', '525225', '779244', '802484', '833668', '755557', '745273', '711380', '866197', '602615', '856750', '893656', '841991', '843618', '857193', '820447', '742211', '772314', '730185', '787513', '871651', '799945', '817662', '737492', '835295', '825577', '833482', '679479', '831236', '870689', '831255', '614263', '822808', '825873', '698340', '842791', '796006', '835189', '793780', '799701', '715467', '800055', '672659', '812718', '823035', '784081', '750514', '858583', '687966', '762086', '670824', '672226', '746464', '791715', '723354', '384708', '761047', '783755', '728787', '793352', '858835', '796761', '722322', '732991', '829057', '587581', '718995', '774131', '728917', '771373', '781766', '827736', '796914', '776623', '729656', '720389', '649218', '785986', '560928', '809924', 

In [23]:
dict_products = {}
for index, row in products.iterrows():
    dict_products[str(row['_id'])] = row['name']

### BASELINE: Wilson score

```
module.exports = function calculateWilsonScore(positiveScore, total) {
  if (total === 0) {
    return {
      lower: 0,
      upper: 0,
    };
  }

  if (positiveScore > total) {
    logger.warn('"positiveScore" cannot be greater than "total"');
    positiveScore = total;
  }

  // phat is the proportion of successes
  // in a Bernoulli trial process
  const phat = positiveScore / total;

  // z is 1-alpha/2 percentile of a standard
  // normal distribution for error alpha=5%
  const z = 1.96;

  // implement the algorithm
  // (http://goo.gl/kgmV3g)
  const a = phat + (z * z) / (2 * total);
  const b = z * Math.sqrt((phat * (1 - phat) + (z * z) / (4 * total)) / total);
  const c = 1 + (z * z) / total;

  return {
    lower: (a - b) / c,
    upper: (a + b) / c,
  };
};
```

In [24]:
used_popularity_insights.head(1)

,_id,ProductId,clicks,impressions,type,createdAt,updatedAt,favorites
27,3356,36194,3,32,historical-ctr,NaN,2022-08-15 10:10:00.654000 +00:00,0


In [25]:
import math
def rec_wilson(top_k=10):
  z = 1.96

  wilson_dict = {}

  for _,p in used_popularity_insights.iterrows():
    pos =  p["favorites"]
    total = p["impressions"]
    phat = pos/total
    a = phat + (z*z)/(2*total)
    b = z*math.sqrt(abs((phat*(1 - phat)+(z*z)/ (4*total))/ total)) # Podemos observar que es despreciable, por lo que basamos nuestro "wilson score" según a.

    wilson_dict[p["ProductId"]] = a

  # print(wilson_dict)
  sorted_wilson = sorted(wilson_dict.items(), key=lambda x: x[1], reverse=True)
  # print(sorted_wilson)
  rec = []
  for i in range(top_k):
    rec.append(sorted_wilson[i][0])
  return rec

print(rec_wilson())


[347166, 634057, 77508, 103584, 116458, 130922, 194833, 206199, 210183, 217593]


## Creación de Matriz de Características

In [26]:
# Primero, vemos cuantos Tags son realmente usados de la lista de tags
used_tags = product_tag.TagId.unique()
used_tags.sort()
# Luego, hacemos una matriz de MxN de 0s donde M son los productos y N son los tags
matrix_dim = (len(used_products),len(used_tags))
tag_matrix = np.zeros(matrix_dim)

# Poblamos la matriz según los productos
tag_id_index = {}
i = 0
for tag in used_tags:
  tag_id_index[tag] = i
  i +=1

product_index = {}
index_product = {}
i = 0 # Nuevo índice basado en la matriz de dimensiones
for _, product in used_products.iterrows():
  product_index[product["_id"]] = i
  index_product[i] = product["_id"]
  i += 1

counter = 0
for _, asc in used_product_tag.iterrows():
  if asc["ProductId"] in product_index.keys() and asc["TagId"] in tag_id_index.keys():
    tag_matrix[product_index[asc["ProductId"]]][tag_id_index[asc["TagId"]]] = 1

# Reducimos dimensionalidad
pca60_tag_matrix = PCA(n_components=60).fit_transform(tag_matrix)

# La pasamos a un DataFrame de Pandas

tag_mtx_df = pd.DataFrame(tag_matrix,
                            index = product_index.keys(),
                            columns = tag_id_index.keys())
pca60_tag_mtx_df = pd.DataFrame(pca60_tag_matrix,
                            index = product_index.keys())




## Prueba de modelos a usar

Random

In [27]:
#Random
import random
def random_rec(n):
  rec = random.sample(lista_popular, n)
  out = []
  out_code = []
  for i in rec:
    out.append(dict_products[str(i[1])])
    out_code.append(i[1])
  return out_code

print(random_rec(5))

[393460, 367704, 713849, 711061, 847611]


Most Popular

In [28]:
# Este most popular es en base a los clicks que se han hecho sobre el producto
# Se pueden incluir los demás insights para hacerlo más preciso

def popular_rec(n=10):
  out = []
  out_code = []
  for i in range(n):
    out.append(dict_products[str(lista_popular[i][1])])
    out_code.append(lista_popular[i][1])
  return out_code

print(popular_rec(5))


[590742, 838042, 849481, 858669, 907085]


Most Popular (Size Sensitive)

In [29]:
def popular_rec_size(prod_list, n=10):
  sizes = []
  p_ids = []
  for p in prod_list:
    if p in tallas.keys():
      sizes.append(tallas[p])

  for key in tallas:
    if tallas[key] in sizes:
      p_ids.append(key)
  
  if len(tallas) != 0:
    popular = used_popularity_insights[used_popularity_insights['ProductId'].isin(p_ids)].head(n)
  else:
    popular = used_popularity_insights.head(n)

  return popular["ProductId"].values.tolist()

  

In [30]:
# popular_rec_size(5, [411235])

Conent Based (simple)

In [49]:
# Esta función encuentra los K productos similares al embedding promedio de una lista de productos
def find_similar_products(embedding, product_id_list, metric='euclidean', top_k=10):
    
    n = embedding.shape[0]

    vec_add = []
    
    idx_list = []
    # print(list(product_index.keys()))
    for p in product_id_list:
      if p in list(product_index.keys()):
        # print('Entré luego del str')
        idx_list.append(product_index[p])

    for idx in idx_list:
      if len(vec_add) == 0:
        vec_add = embedding[idx]
      else:
        vec_add = np.add(vec_add, embedding[idx])

    if len(vec_add) == 0:
      return []
    
    vec_add = np.divide(vec_add, len(idx_list))
    distances = pairwise_distances(vec_add.reshape(1,-1), embedding, metric=metric)
    heap = []
    for i in range(n):            
        if len(heap) < top_k*5:
            heapq.heappush(heap, (-distances[0][i], i))
        else:
            heapq.heappushpop(heap, (-distances[0][i], i))

    heap.sort(reverse=True)
    rec_ids = [index_product[i] for _,i in heap]
    
    return random.sample(rec_ids, top_k)

In [32]:
tag_mtx_df

,13,25,26,27,30,31,32,34,36,40,...,1105,1106,1107,1110,1113,1114,1115,1116,1117,1118
26367,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
36194,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
36634,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
46336,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
52350,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
987831,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
988112,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
988558,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
988855,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [33]:
# id_list = find_similar_products(pca60_tag_matrix, [984926])
# print(list(product_index.keys()))
id_list = find_similar_products(pca60_tag_matrix, random.sample(list(product_index.keys()), 10))

In [34]:
used_products[used_products['_id'].isin(id_list)]

,_id,price,referencePrice,name,BrandId,customAttributes,color,pattern,detailsScore,composition,storeDescription
1026,237013,5090,52990,Vestido Topshop 38 Verde,695,"{""2"":"""",""3"":148,""5"":0,""7"":45,""9"":"""",""113"":"""",""...",green,none,1,"[{""material"":""viscosa"",""percentage"":87},{""mate...",Marca: Topshop\nTalla: EUR 38\nMedidas: Largo ...
1775,284986,2995,15990,Falda H&M M Negro,721,"{""3"":41,""8"":42,""11"":48,""136"":""2""}",black,none,1,"[{""material"":""algodón"",""percentage"":95},{""mate...",Marca: H&M\nTalla: EUR M\nMedidas: Largo (cms)...
2059,301173,7990,21990,Falda Candie's S Azul,2189,"{""3"":42,""8"":42,""11"":27,""136"":""2""}",blue,none,1,"[{""material"":""viscosa"",""percentage"":94},{""mate...",Marca: Candie's\nTalla: S\nMedidas: Largo (cms...
16755,659168,4990,12990,Body Forever 21 M,680,{},NaN,NaN,1,"[{""material"":""poliéster"",""percentage"":100}]",Marca: Forever 21\nTalla: EUR M\nMedidas:\nCon...
24049,773332,42990,134990,Anteojos Ray-Ban,1843,"{""136"":""2""}",NaN,NaN,2,[],Marca: Ray-Ban\nMedidas:\nCondición: Con detal...
27456,808048,12990,34990,Enterito Mango L,1777,"{""3"":151,""5"":1,""7"":51,""136"":""2""}",NaN,NaN,2,[],Marca: Mango\nTalla: EUR L\nMedidas: Largo (cm...
29538,832065,3390,10170,Cinturón Sin Marca,11647,{},NaN,NaN,2,[],Marca: Sin Marca\nMedidas:\nCondición: Con det...
31900,860347,3390,7890,Pañuelo Sin Marca,11647,"{""3"":110,""136"":""2""}",NaN,NaN,2,[],Marca: Sin Marca\nMedidas: Largo (cms): 110\nC...
32840,870652,4490,11190,Pañuelo FES,6774,"{""3"":""179"",""136"":""2""}",NaN,NaN,1,[],Marca: FES\nMedidas: Largo (cms): 179\nCondici...
34219,888025,4990,14990,Gorro O´Neill,2730,"{""136"":""2""}",NaN,NaN,2,"[{""material"":""acrílico"",""percentage"":100}]",Marca: O´Neill\nMedidas:\nCondición: Con detal...


Content Based (mix)

In [35]:
# Esta función encuentra los K productos similares a m promedios obtenidos de una lista de productos
def find_similar_products_mix(embedding, product_id_list, metric='euclidean', top_k=10, meansm = 3):
  sample_size = max(round(len(product_id_list)/4), 1)
  samples = []
  for _ in range(meansm):
    samples.append(random.sample(product_id_list, sample_size))

  recomendations = []
  for s in samples:
    recomendations += find_similar_products(pca60_tag_matrix, s)

  recomendations = list( dict.fromkeys(recomendations) )
  return random.sample(recomendations, min(top_k, len(recomendations)))

In [36]:
used_products.sample(10)

,_id,price,referencePrice,name,BrandId,customAttributes,color,pattern,detailsScore,composition,storeDescription
25739,790158,9990,19990,Vestido H&M 46,721,"{""3"":126,""5"":1,""7"":50,""136"":""2"",""212"":""19.990""}",NaN,NaN,0,"[{""material"":""algodón"",""percentage"":100}]",Marca: H&M\nTalla: EUR 46\nMedidas: Largo (cms...
29187,828107,7790,21290,Vestido H&M S,721,"{""3"":74,""5"":1,""7"":50,""136"":""2""}",NaN,NaN,0,"[{""material"":""viscosa"",""percentage"":100}]",Marca: H&M\nTalla: EUR S\nMedidas: Largo (cms)...
17408,673947,2990,6990,Pañuelo Sin Marca,11647,"{""3"":88,""136"":""2""}",NaN,NaN,0,[],Marca: Sin Marca\nMedidas: Largo (cms): 88\nCo...
27619,809834,8990,24590,Blusa ASH,8389,"{""3"":69,""7"":55,""136"":""2""}",NaN,NaN,0,[],Marca: ASH\nTalla: 44\nMedidas: Largo (cms): 6...
9355,532681,4490,11190,Polera Opposite Puntos,1528,"{""3"":51,""5"":1,""7"":43,""136"":""2""}",black,dots,0,"[{""material"":""viscosa"",""percentage"":100}]",Marca: Opposite\nTalla: S\nMedidas: Largo (cms...
26324,796459,6620,25990,Body Bershka S,1700,{},NaN,NaN,3,[],Marca: Bershka\nTalla: EUR S\nMedidas:\nCondic...
28712,822769,8990,24590,Pescador H&M 42,721,"{""3"":95,""11"":43,""12"":30,""136"":""2""}",NaN,NaN,0,[],Marca: H&M\nTalla: EUR 42\nMedidas: Largo (cms...
22187,751231,7990,21990,Blusa Estam S,6748,"{""3"":52,""7"":46,""136"":""2""}",NaN,NaN,0,"[{""material"":""poliéster"",""percentage"":95},{""ma...",Marca: Estam\nTalla: S\nMedidas: Largo (cms): ...
4184,386720,7990,22990,Camisa Aeropostale M Naranjo Cuadros,1535,"{""3"":61,""5"":41,""7"":53,""136"":""2""}",orange,checkered,1,[],Marca: Aeropostale\nTalla: M\nMedidas: Largo (...
6887,472587,3000,21290,Chaleco Perry Ellis,2181,"{""3"":61,""5"":1,""7"":44,""136"":""1""}",brown,none,1,"[{""material"":""viscosa"",""percentage"":72},{""mate...",Marca: Perry Ellis\nTalla: 2\nMedidas: Largo (...


In [37]:
# find_similar_products_mix(pca60_tag_matrix, [693643, 489690, 824508, 767328, 799826, 686343, 790180, 769585, 880790, 856164])
# print(list(used_products.sample(10).iloc[:, 0]))
find_similar_products_mix(pca60_tag_matrix, list(used_products.sample(10).iloc[:, 0]))


[317863,
 307537,
 758838,
 780551,
 859332,
 807296,
 798603,
 444734,
 584240,
 684807]

## Definición de situaciones

In [59]:
# En base a los datos obtenidos, se define una función para hacer el switching y seleccionar
# el modelo acorde a lo necesitado

# Definición básica de situaciones

def switch_rec(n_transactions, user_id, top_k=10):
  prod_len = 0
  products = []
  if user_id in dict_sales.keys():
    prod_len = len(dict_sales[user_id])
    products = dict_sales[user_id]
    for i in range(0, len(products)):
      products[i] = int(products[i])
  # print(products)
  if n_transactions == 0:
    return random_rec(top_k)
  else:
    if prod_len == 0:
      return popular_rec(top_k)
    elif prod_len < 5:
      return popular_rec_size(products, n=top_k)
    elif prod_len < 11:
      return find_similar_products(pca60_tag_matrix, products)
    else:
      return find_similar_products_mix(pca60_tag_matrix, products)




## Implementación de sistema recomendador

In [39]:
# Funciones que llaman a switching de acuerdo a la cantidad de transacciones que
# existan. En este caso se hardcodean 3 casos distintos.

# print(switch_rec(10))
# print(switch_rec(500))
# print(switch_rec(2000))

## Evaluación

Métrica custom de error:

Para todo usuario parte del test:
- Se toma un sample de tamaño n random entre 2 y 10 de sus productos.
- Se genera una predicción con el switch a partir de este sample
- Si algún item de esta predicción coincide con alguno de los otros productos comprados por el usuario, se considera un éxito

Se retorna exitos/totales*100

In [51]:
def prediction_correctness(person_id, algorithm='content_based', top_k=10):
  s_size = random.randint(2,5)
  user_products = [ int(x) for x in dict_sales[person_id] ]
  query_sample = random.sample(user_products,s_size)
  if algorithm == 'content_based':
    recommended = find_similar_products(pca60_tag_matrix, query_sample)
  elif algorithm == 'content_based_mix':
    recommended = find_similar_products_mix(pca60_tag_matrix, query_sample)
  elif algorithm == 'popular_rec_size':
    recommended = popular_rec_size(user_products)
  elif algorithm == 'wilson':
    recommended = rec_wilson()
  else:
    recommended = popular_rec()
  success = False
  for r in recommended:
    if r in user_products and r not in query_sample:
      success = True

  return success

def success_rate(id_list, algorithm='content_based', top_k = 10):
  suc = 0
  for id in id_list:
    if prediction_correctness(id, algorithm=algorithm, top_k=top_k):
      suc += 1
  return suc/len(id_list)*100


#### Correctness

Switch

In [52]:
# Sacamos 10 samples y revisamos su correctitud con cada uno de los modelos. Sacamos un promedio por cada uno y ese lo tomamos como el success-percentage
pop_prc = 0
pop_s_prc = 0
c_b_prc = 0
c_b_m_prc = 0
for i in range(10):
  sample = random.sample(list(used_clients), min(100, len(used_clients)))

  pop_prc += success_rate(sample, algorithm='popular_rec')
  pop_s_prc += success_rate(sample, algorithm='popular_rec_size')
  c_b_prc += success_rate(sample, algorithm='content_based')
  c_b_m_prc += success_rate(sample, algorithm='content_based_mix')

print(f"Popular Success percent:           {pop_prc/10}%")
print(f"Popular Size success percent:      {pop_s_prc/10}%")
print(f"Content Based success percent:     {c_b_prc/10}%")
print(f"Content Based Mix success percent: {c_b_m_prc/10}%")
#prediction_correctness(24120)

Popular Success percent:           0.9%
Popular Size success percent:      12.8%
Content Based success percent:     14.5%
Content Based Mix success percent: 16.4%


Switch (20)

In [53]:
# Sacamos 10 samples y revisamos su correctitud con cada uno de los modelos. Sacamos un promedio por cada uno y ese lo tomamos como el success-percentage
pop_prc = 0
pop_s_prc = 0
c_b_prc = 0
c_b_m_prc = 0
for i in range(10):
  sample = random.sample(list(used_clients), min(100, len(used_clients)))

  pop_prc += success_rate(sample, algorithm='popular_rec', top_k=20)
  pop_s_prc += success_rate(sample, algorithm='popular_rec_size', top_k=20)
  c_b_prc += success_rate(sample, algorithm='content_based', top_k=20)
  c_b_m_prc += success_rate(sample, algorithm='content_based_mix', top_k=20)

print(f"Popular Success percent:           {pop_prc/10}%")
print(f"Popular Size success percent:      {pop_s_prc/10}%")
print(f"Content Based success percent:     {c_b_prc/10}%")
print(f"Content Based Mix success percent: {c_b_m_prc/10}%")

Popular Success percent:           1.0%
Popular Size success percent:      13.3%
Content Based success percent:     14.9%
Content Based Mix success percent: 16.2%


Switch (30)

In [57]:
# Sacamos 10 samples y revisamos su correctitud con cada uno de los modelos. Sacamos un promedio por cada uno y ese lo tomamos como el success-percentage
pop_prc = 0
pop_s_prc = 0
c_b_prc = 0
c_b_m_prc = 0
for i in range(10):
  sample = random.sample(list(used_clients), min(100, len(used_clients)))

  pop_prc += success_rate(sample, algorithm='popular_rec', top_k=30)
  pop_s_prc += success_rate(sample, algorithm='popular_rec_size', top_k=30)
  c_b_prc += success_rate(sample, algorithm='content_based', top_k=30)
  c_b_m_prc += success_rate(sample, algorithm='content_based_mix', top_k=30)

print(f"Popular Success percent:           {pop_prc/10}%")
print(f"Popular Size success percent:      {pop_s_prc/10}%")
print(f"Content Based success percent:     {c_b_prc/10}%")
print(f"Content Based Mix success percent: {c_b_m_prc/10}%")

Popular Success percent:           0.9%
Popular Size success percent:      11.9%
Content Based success percent:     14.9%
Content Based Mix success percent: 16.8%


Wilson

In [42]:
rec_wilson()
print(f"Wilson Score Success percent:           {success_rate(random.sample(list(used_clients), min(100, len(used_clients))), algorithm='wilson')}%")

Wilson Score Success percent:           2.0%


Wilson (20)

In [48]:
rec_wilson(top_k=20)
print(f"Wilson Score (20) Success percent: {success_rate(random.sample(list(used_clients), min(100, len(used_clients))), algorithm='wilson')}%")

Wilson Score (20) Success percent: 2.0%


Wilson (30)

In [56]:
rec_wilson(top_k=30)
print(f"Wilson Score (30) Success percent: {success_rate(random.sample(list(used_clients), min(100, len(used_clients))), algorithm='wilson')}%")

Wilson Score (30) Success percent: 3.0%


#### Intra-list Similarity

Switch

In [43]:
used_client = random.sample(list(used_clients), 1)[0]
print(used_client)

id_list = switch_rec(10, used_client)

98910


In [44]:
intra_list_similarity = 0
for i in range(25):
  recommended = used_products[used_products['_id'].isin(id_list)][["_id"]].values
  recommended = [recommended[r][0] for r in range(len(recommended))]
  intra_list_similarity += recmetrics.intra_list_similarity([recommended, recommended], tag_mtx_df)

print(f"Intra list similarity: {intra_list_similarity/25}")

Intra list similarity: 0.30488961174028445


Switch(20)

In [60]:
used_client = random.sample(list(used_clients), 1)[0]
print(used_client)

id_list = switch_rec(10, used_client, top_k=20)

98956


In [61]:
intra_list_similarity = 0
for i in range(25):
  recommended = used_products[used_products['_id'].isin(id_list)][["_id"]].values
  recommended = [recommended[r][0] for r in range(len(recommended))]
  intra_list_similarity += recmetrics.intra_list_similarity([recommended, recommended], tag_mtx_df)

print(f"Intra list similarity: {intra_list_similarity/25}")

Intra list similarity: 0.3895500050021903


Switch(30)

In [62]:
id_list = switch_rec(10, used_client, top_k=30)
intra_list_similarity = 0
for i in range(25):
  recommended = used_products[used_products['_id'].isin(id_list)][["_id"]].values
  recommended = [recommended[r][0] for r in range(len(recommended))]
  intra_list_similarity += recmetrics.intra_list_similarity([recommended, recommended], tag_mtx_df)

print(f"Intra list similarity: {intra_list_similarity/25}")

Intra list similarity: 0.29845576188567163


Wilson

In [45]:
id_list = rec_wilson()
intra_list_similarity = 0
for i in range(25):
  recommended = used_products[used_products['_id'].isin(id_list)][["_id"]].values
  recommended = [recommended[r][0] for r in range(len(recommended))]
  intra_list_similarity += recmetrics.intra_list_similarity([recommended, recommended], tag_mtx_df)

print(f"Intra list similarity: {intra_list_similarity/25}")

Intra list similarity: 0.08305167439565306


Wilson(20)

In [63]:
id_list = rec_wilson(top_k=20)
intra_list_similarity = 0
for i in range(25):
  recommended = used_products[used_products['_id'].isin(id_list)][["_id"]].values
  recommended = [recommended[r][0] for r in range(len(recommended))]
  intra_list_similarity += recmetrics.intra_list_similarity([recommended, recommended], tag_mtx_df)

print(f"Intra list similarity: {intra_list_similarity/25}")

Intra list similarity: 0.10325156264937395


Wilson(30)

In [64]:
id_list = rec_wilson(top_k=30)
intra_list_similarity = 0
for i in range(25):
  recommended = used_products[used_products['_id'].isin(id_list)][["_id"]].values
  recommended = [recommended[r][0] for r in range(len(recommended))]
  intra_list_similarity += recmetrics.intra_list_similarity([recommended, recommended], tag_mtx_df)

print(f"Intra list similarity: {intra_list_similarity/25}")

Intra list similarity: 0.11127395431462382


#### Personalization

Switch

In [46]:
recs = []
for i in dict_sales.keys():
  recs.append(switch_rec(10, i))

recmetrics.personalization(recs)

0.9597622027534418

Switch(20)

In [65]:
recs = []
for i in dict_sales.keys():
  recs.append(switch_rec(10, i, top_k=20))

recmetrics.personalization(recs)

0.9651097963363295

Switch(30)

In [66]:
recs = []
for i in dict_sales.keys():
  recs.append(switch_rec(10, i, top_k=30))

recmetrics.personalization(recs)

0.9681533735351007

Wilson

In [47]:
recs = []
for i in dict_sales.keys():
  recs.append(rec_wilson())

recmetrics.personalization(recs)

2.220446049250313e-16